### Interactive Visual Analytics with Folium

In [3]:
pip install folium pandas


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Load dataset from URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
df = pd.read_csv(url)
df.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


###  Mark all launch sites on a map

In [5]:
# Extract unique launch site names and their locations
launch_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Create a folium Map centered at average coordinates
map_center = [launch_sites['Latitude'].mean(), launch_sites['Longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Add a marker for each launch site
for i, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Display the map
site_map


###  Coordinates For Each Site.

In [6]:
# Select relevant columns
site_coords = df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]

# Display first few rows
site_coords.head()

,LaunchSite,Latitude,Longitude,Class
0,CCAFS SLC 40,28.561857,-80.577366,0
1,CCAFS SLC 40,28.561857,-80.577366,0
2,CCAFS SLC 40,28.561857,-80.577366,0
3,VAFB SLC 4E,34.632093,-120.610829,0
4,CCAFS SLC 40,28.561857,-80.577366,0


### Start location is NASA Johnson Space Center

In [8]:
import folium

# Start location is NASA Johnson Space Center at Houston, Texas
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create folium Map object centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Display the map
site_map


### We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate.

In [9]:
# Add a circle with a popup label
folium.Circle(
    location=nasa_coordinate,
    radius=500,              # radius in meters
    color='blue',
    fill=True,
    fill_color='blue'
).add_to(site_map)

# Add a text label (popup)
folium.Marker(
    location=nasa_coordinate,
    popup='NASA Johnson Space Center',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Display the map
site_map


### Create and add folium.Circle and folium.Marker for each launch site on the site map

In [10]:
# Loop through each launch site row
for _, row in launch_sites.iterrows():
    site_name = row['LaunchSite']
    coordinate = [row['Latitude'], row['Longitude']]
    
    # Add a circle
    folium.Circle(
        location=coordinate,
        radius=500,           # in meters
        color='#000000',      # black circle border
        fill=True,
        fill_color='#3186cc'
    ).add_child(
        folium.Popup(site_name)
    ).add_to(site_map)
    
    # Add a text label marker
    folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{site_name}</b></div>',
        )
    ).add_to(site_map)


In [11]:
site_map


### Mark the success/failed launches for each site on the map
- Color indicates Class:
     - Success (Class = 1) = green marker
     - Failure (Class = 0) = red marker

In [12]:
from folium.plugins import MarkerCluster

# Create map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Create cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Loop and add markers to the cluster
for _, row in df.iterrows():
    coordinate = [row['Latitude'], row['Longitude']]
    site_name = row['LaunchSite']
    orbit = row['Orbit']
    launch_success = 'Success' if row['Class'] == 1 else 'Failure'
    
    marker_color = 'green' if row['Class'] == 1 else 'red'
    
    folium.Marker(
        location=coordinate,
        popup=folium.Popup(f"Site: {site_name}<br>Orbit: {orbit}<br>Outcome: {launch_success}", max_width=200),
        icon=folium.Icon(color=marker_color, icon='info-sign')
    ).add_to(marker_cluster)

# Show map
site_map


### Calculate the distances between a launch site to its proximities

In [13]:
from folium.plugins import MousePosition

# Create the map centered somewhere reasonable (e.g., NASA)
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Add MousePosition plugin
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" | ",
    empty_string="",
    lng_first=True,
    num_digits=5,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Show the map
site_map


### Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [15]:
# Coordinates
launch_site_lat = 28.5618571
launch_site_lon = -80.577366

coastline_lat = 28.563209
coastline_lon = -80.568843

#function
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Calculate distance
distance_km = calculate_distance(
    launch_site_lat,
    launch_site_lon,
    coastline_lat,
    coastline_lon
)

print(f"Distance from Launch Site to Coastline: {distance_km:.2f} km")


Distance from Launch Site to Coastline: 0.85 km


### Draw a PolyLine between a launch site to the selected coastline point

In [16]:
import folium

# Define coordinates
launch_site_coord = [28.5618571, -80.577366]
coastline_coord = [28.563209, -80.568843]

# Create map centered on the launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=15)

# Add markers
folium.Marker(
    location=launch_site_coord,
    popup="Launch Site",
    icon=folium.Icon(color="blue")
).add_to(site_map)

folium.Marker(
    location=coastline_coord,
    popup="Coastline",
    icon=folium.Icon(color="green")
).add_to(site_map)

# Draw PolyLine
folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    color='red',
    weight=3,
    opacity=0.8
).add_to(site_map)

# Show the map
site_map


### Draw a line betwee a launch site to its closest city, railway, highway, etc.Use MousePosition to find the their coordinates on the map first



In [18]:
# Launch site
launch_site_coord = [28.5618571, -80.577366]

# Nearby features
railway_coord = [28.562302, -80.571766]
highway_coord = [28.563972, -80.570662]
coastline_coord = [28.563209, -80.568843]
city_coord = [28.608197, -80.604138]  # Example: Cocoa Beach

In [19]:
import folium

# Initialize the map centered on the launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=12)

# Add launch site marker
folium.Marker(
    location=launch_site_coord,
    popup="Launch Site",
    icon=folium.Icon(color="blue")
).add_to(site_map)

# Add markers for each feature
folium.Marker(
    location=railway_coord,
    popup="Railway",
    icon=folium.Icon(color="gray")
).add_to(site_map)

folium.Marker(
    location=highway_coord,
    popup="Highway",
    icon=folium.Icon(color="orange")
).add_to(site_map)

folium.Marker(
    location=coastline_coord,
    popup="Coastline",
    icon=folium.Icon(color="green")
).add_to(site_map)

folium.Marker(
    location=city_coord,
    popup="Closest City",
    icon=folium.Icon(color="purple")
).add_to(site_map)


In [20]:
# Line to Railway
folium.PolyLine(
    locations=[launch_site_coord, railway_coord],
    color='red',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Line to Highway
folium.PolyLine(
    locations=[launch_site_coord, highway_coord],
    color='red',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Line to Coastline
folium.PolyLine(
    locations=[launch_site_coord, coastline_coord],
    color='red',
    weight=2,
    opacity=0.8
).add_to(site_map)

# Line to City
folium.PolyLine(
    locations=[launch_site_coord, city_coord],
    color='red',
    weight=2,
    opacity=0.8
).add_to(site_map)


In [22]:
site_map
